In [18]:
import pyscal.core as pc
import pyscal.crystal_structures as pcs
import os
import numpy as np
from tqdm import tqdm

In [19]:
from pymolsim import Sim, Thermostat, Barostat, Particle
from pymolsim.potential import LJ

In [20]:
sys = pc.System()
sys.read_inputfile("conf.dump", customkeys=["mass", "vx", "vy", "vz"])
atoms = sys.atoms
box = sys.box
particles = []
for atom in atoms:
    p = Particle()
    p.mass = 1.00;
    p.r = np.array(atom.pos);
    p.v = np.array([float(atom.custom["vx"]), float(atom.custom["vy"]), float(atom.custom["vz"])])
    p.type = atom.type
    particles.append(p)

In [21]:
sim = Sim(particles, box)

In [22]:
sim.beta = 0.5
sim.pressure = 0
sim.dt = 0.0025
sim.nthreads = 4

Create a potential

In [23]:
lj = LJ(1.0, 1.0)

Attch the potential to syatem

In [24]:
sim.potential = lj

We can start md now

In [8]:
sim.start()

In [9]:
sim.forces()

In [12]:
fs = []
particles = sim.particles
for par in particles:
    fs.append(par.f)

In [15]:
np.savetxt("forces_test.dat", fs)

In [10]:
%%timeit
sim.forces()

20.7 s ± 428 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
sim.forces()

3.68 s ± 55.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
sim.forces()
sim.remap()

In [10]:
sim.dump(0)

In [11]:
for i in tqdm(range(1, 10)):
    sim.md_verlet()
    sim.remap()
    sim.dump(i)

100%|██████████| 9/9 [00:34<00:00,  3.84s/it]


In [25]:
from joblib import Parallel, delayed
import time

In [15]:
def addnum(i, j):
    time.sleep(0.01)
    return i+j

In [16]:
calcs = []
for i in range(10):
    for j in range(10):
        calcs.append(delayed (addnum) (i,j))

In [21]:
%%timeit
r = Parallel(n_jobs=4)(calcs)

274 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
100*0.01

1.0

In [27]:
a =  np.array([np.array([1,1,1]), 
               np.array([2,2,2]), 
               np.array([3,3,3])])

In [28]:
x = np.array([0.0, 0.5, 0.0, 0.0])
y = np.array([0.0, 0.0, 0.5, 0.0])
z = np.array([0.0, 0.0, 0.0, 0.5])
o = np.array([0.0, 1.0, 0.0, 0.0])

In [17]:
xx = np.zeros((len(x), len(x)))
for i in range(len(x)):
    for j in range(len(x)):
        xx[i, j] = x[i] - x[j]

NameError: name 'x' is not defined

In [74]:
xx

array([[ 0. , -0.5,  0. ,  0. ],
       [ 0.5,  0. ,  0.5,  0.5],
       [ 0. , -0.5,  0. ,  0. ],
       [ 0. , -0.5,  0. ,  0. ]])

In [29]:
xd = np.meshgrid(x, x)[1] - np.meshgrid(x, x)[0]
yd = np.meshgrid(y, y)[1] - np.meshgrid(y, y)[0]
zd = np.meshgrid(z, z)[1] - np.meshgrid(z, z)[0]

In [34]:
np.meshgrid(x, x)[0]

array([[0. , 0.5, 0. , 0. ],
       [0. , 0.5, 0. , 0. ],
       [0. , 0.5, 0. , 0. ],
       [0. , 0.5, 0. , 0. ]])

In [43]:
x

array([0. , 0.5, 0. , 0. ])

In [41]:
def fsum(x, y, z):
    r = (x**2 + y**2 + z**2)**0.5
    f = -1*r
    fx = f*x
    fy = f*y
    fz = f*z
    return fx, fy, fz

In [45]:
fx, fy, fz = fsum(xd, yd, zd)

In [64]:
np.sum(fx[0]), np.sum(fx[1]), np.sum(fx[2]), np.sum(fx[3])

(0.25, -0.9571067811865475, 0.3535533905932738, 0.3535533905932738)

In [66]:
np.sum(fx, axis=-1)

array([ 0.25      , -0.95710678,  0.35355339,  0.35355339])

In [73]:
x

array([0. , 0.5, 0. , 0. ])

In [76]:
np.where(x>0.4)

(array([1]),)

In [86]:
a = np.array([[0, 1, 2],
              [0, 2, 4],
              [0, 3, 6]])
b = np.array([[0, 1, 2],
              [0, 1, 2],
              [0, 1, 2]])

In [88]:
np.where(a > 4, b*2, b)

array([[0, 1, 2],
       [0, 1, 2],
       [0, 1, 4]])